In [ ]:
import sys
sys.path.append('On_Multiple_Semantics_for_Declarative_Database_Repairs/')
import On_Multiple_Semantics_for_Declarative_Database_Repairs

from database_generator.dba import DatabaseEngine
from Semantics.end_sem import EndSemantics
from Semantics.stage_sem import StageSemantics
from Semantics.step_sem import StepSemantics
from Semantics.independent_sem import IndependentSemantics

# specify the schema
mas_schema = {"synthetic_data_version_1": ('RAC1P', 'SEX', 'REGION', 'PINCP', 'ST', 'COW', 'CIT', 'NATIVITY', 'MSP', 'SCHL', 'DIS','MIL')
              }


def read_rules(rule_file):
    """read programs from txt file"""
    all_programs = []
    with open(rule_file) as f:
        rules = []
        for line in f:
            if line.strip():
                tbl, r = line.split("|")
                rules.append((tbl, r[:-2]))
            else:
                all_programs.append([r for r in rules])
                rules = []
        all_programs.append(rules)
    return all_programs

# load delta programs
programs = read_rules("On_Multiple_Semantics_for_Declarative_Database_Repairs/../delta_program.txt")

# start the database
db = DatabaseEngine("cr")

tbl_names = ["synthetic_data_version_1"]
subfolder = "/datasets/muse/"

def database_reset(db):
    """reset the database"""
    res = db.execute_query("select current_database();")
    db_name = res[0][0]
    db.delete_tables(tbl_names)
    # db.close_connection()
    db = DatabaseEngine(db_name)
    db.load_database_tables(tbl_names)

# choose a delta program from the programs file
rules = programs[0]
print("delta program:", rules)

# reset the database between runs
database_reset(db)

stage_sem = StageSemantics(db, rules, tbl_names)
stage_semantics_result = stage_sem.find_mss()
print("result for stage semantics:", stage_semantics_result)


delta program: [('synthetic_data_version_1', 'SELECT synthetic_data_version_11.* FROM synthetic_data_version_1 AS synthetic_data_version_11, synthetic_data_version_1 AS synthetic_data_version_12 WHERE synthetic_data_version_11.ST = synthetic_data_version_12.ST AND synthetic_data_version_11.REGION <> synthetic_data_version_12.REGION')]


In [ ]:
len(stage_semantics_result)